In [1]:

import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
import os.path as osp
import pandas as pd
import numpy as np
import collections
from pandas.core.common import flatten
# importing obg datatset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from pandas.core.common import flatten
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(16.7,8.27)})
sns.set_theme(style="ticks")
import collections
from scipy.special import softmax
import umap

/home/ljb/miniconda3/envs/ai/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ljb/miniconda3/envs/ai/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ljb/miniconda3/envs/ai/lib/python3.11/site-packages/umap/distances.py:1086: 

In [2]:
from pathlib import Path
import numpy as np
from rich import print
DATA_ROOT = Path('kaggle_data')

print('Loading Train data...')
ndf_files_train = list((DATA_ROOT / 'npz_all/npz/layout/nlp/default/train').iterdir())
nrd_files_train = list((DATA_ROOT / 'npz_all/npz/layout/nlp/random/train').iterdir())
xdf_files_train = list((DATA_ROOT / 'npz_all/npz/layout/xla/default/train').iterdir())
xrd_files_train = list((DATA_ROOT / 'npz_all/npz/layout/xla/random/train').iterdir())

ndf_nps_train = [np.load(f) for f in ndf_files_train]
nrd_nps_train = [np.load(f) for f in nrd_files_train]
xdf_nps_train = [np.load(f) for f in xdf_files_train]
xrd_nps_train = [np.load(f) for f in xrd_files_train]

print('Loading Validation data...')
ndf_files_valid = list((DATA_ROOT / 'npz_all/npz/layout/nlp/default/valid').iterdir())
nrd_files_valid = list((DATA_ROOT / 'npz_all/npz/layout/nlp/random/valid').iterdir())
xdf_files_valid = list((DATA_ROOT / 'npz_all/npz/layout/xla/default/valid').iterdir())
xrd_files_valid = list((DATA_ROOT / 'npz_all/npz/layout/xla/random/valid').iterdir())

ndf_nps_valid = [np.load(f) for f in ndf_files_valid]
nrd_nps_valid = [np.load(f) for f in nrd_files_valid]
xdf_nps_valid = [np.load(f) for f in xdf_files_valid]
xrd_nps_valid = [np.load(f) for f in xrd_files_valid]

print('Loading Test data...')
ndf_files_test = list((DATA_ROOT / 'npz_all/npz/layout/nlp/default/test').iterdir())
nrd_files_test = list((DATA_ROOT / 'npz_all/npz/layout/nlp/random/test').iterdir())
xdf_files_test = list((DATA_ROOT / 'npz_all/npz/layout/xla/default/test').iterdir())
xrd_files_test = list((DATA_ROOT / 'npz_all/npz/layout/xla/random/test').iterdir())

ndf_nps_test = [np.load(f) for f in ndf_files_test]
nrd_nps_test = [np.load(f) for f in nrd_files_test]
xdf_nps_test = [np.load(f) for f in xdf_files_test]
xrd_nps_test = [np.load(f) for f in xrd_files_test]

Loading Train data...

Loading Validation data...

Loading Test data...

In [3]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3):
        super(SAGE, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.
        layer_1_embeddings, layer_2_embeddings, layer_3_embeddings = None, None, None
        for i, (edge_index, _, size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
            xs.append(x)
            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all    
        #return x.log_softmax(dim=-1)
        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x)

                pbar.update(batch_size)

            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
                
        pbar.close()

        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings
def aggregate_outputs(out):
    # Example: simple average
    return sum(out) / len(out)

class myModel(torch.nn.Module):
    def  __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.SAGE = SAGE(*args)
        self.config_fc = torch.nn.Linear(88 * 18, 64)  # Flatten and map to 64-dim
        self.final_fc = torch.nn.Linear(128, 1)

    def forward(self, x, adjs, config_feat):
        out = self.SAGE(x, adjs)
        for o in out:
            print(o.shape)
        aggregated_sage_output = aggregate_outputs(out)
        print(aggregated_sage_output.shape)
        config_feat  = config_feat.view(1000, -1)
        processed_config_feat = self.config_fc(config_feat)
        combined_feat = torch.cat([aggregated_sage_output, processed_config_feat], dim=1)
        final_output = self.final_fc(combined_feat)
        return final_output



In [4]:
def check_npz_shape(npz_file):
    keys_and_shapes = {key: npz_file[key].shape for key in npz_file.keys()}
    print(keys_and_shapes)
check_npz_shape(ndf_nps_train[0])

{
    'node_feat': (1696, 140),
    'node_opcode': (1696,),
    'edge_index': (2697, 2),
    'node_config_feat': (100040, 121, 18),
    'node_config_ids': (121,),
    'config_runtime': (100040,),
    'node_splits': (1, 2)
}

In [5]:
from torch_geometric.loader import NeighborSampler

dataset = xdf_nps_train #+ xrd_nps_train + ndf_nps_train + nrd_nps_train
device = 'cpu'
# 初始化NeighborSampler
sampler = NeighborSampler(
    edge_index=torch.tensor(dataset[0]['edge_index']),
    sizes=[15, 10, 5],  # 这里的sizes是一个列表，表示每一层的邻居采样数
    batch_size=2000,  # 每个批次中的节点数
    shuffle=True,    # 是否打乱节点
    num_workers=0    # 数据加载的并行度
)
# Initialize the model
in_channels = 140
hidden_channels = 64  # This is just an example, can be tuned
out_channels = 1  # This can be tuned based on the specific task
num_layers = 3

model = myModel(in_channels, hidden_channels, out_channels, num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

# 数据迭代和模型训练
def train(epoch):
    for data in dataset:  # Assume `dataset` is an iterable of graph data dictionaries
        x = torch.tensor(data['node_feat'], dtype=torch.float).to(device)  # 假设device是你的计算设备（CPU或GPU）
        edge_index = torch.tensor(data['edge_index'], dtype=torch.long).to(device)
        config_feat = torch.tensor(data['node_config_feat'], dtype=torch.float).to(device)
        print(config_feat.shape)
        # 更新NeighborSampler的edge_index
        sampler.edge_index = edge_index

        # Forward and backward passes
        for batch_size, n_id, adjs in sampler:
            # adjs已经是一个列表，其中包含了多跳邻居的信息
            
            # 对于每个批次，获取相应的节点特征
            x_batch = x[n_id].to(device)  # n_id是这个批次中所有节点的id

            # 前向传播
            out = model(x_batch, adjs, config_feat)
            break
        break
            # 计算损失、反向传播、更新等（这取决于你的具体任务和损失函数）
            # ...
    return None

train(1)

torch.Size([47712, 26, 18])

torch.Size([372, 64])

torch.Size([372, 64])

torch.Size([372, 1])

torch.Size([372, 64])

RuntimeError: shape '[1000, -1]' is invalid for input of size 22329216